In [299]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import seaborn as sns

Problem: Patients not moving correctly from service to served. Noticed because `n_arrived < n_served`. Verified by manually checking `n_arrivals` against `n_served` in `check_in` step from trial where saw issue. Further noticed that issue is in moving from `service` to `completed` when `n_arrivals > n_servers` and wait time is created. 

Specifically, issue is of format (2 examples): 

| Arrivals | Service | Waiting | Completed |
| --- | --- | --- | --- |
| 7 | 5 | 2 | 0 |
| 0 | 5 | 0 | 3 |
| \\ | \\ | \\ | \\ |
| 6 | 5 | 1 | 0 |
| 0 | 3 | 0 | 4 |

When the table should look like:

| Arrivals | Service | Waiting | Completed |
| --- | --- | --- | --- |
| 7 | 5 | 2 | 0 |
| 0 | 4 | 0 | 3 |
| \\ | \\ | \\ | \\ |
| 6 | 5 | 1 | 0 |
| 0 | 2 | 0 | 4 |

Data for running simul re-stated, not edited

In [300]:
# Estimates for how often the worst case scenario in patient service times occurs
low = 0.05
medium = 0.1
high = 0.2

In [301]:
# Providers available
providers = {'Doctor': 9, 'Nurse': 5, 'FlowStaff': 20, 'CSR': 10}

In [302]:
cols = ['Step', 'Process', 'Staff', 'Time_Mean', 'Time_WorstCase', 'Perc_WorstCase']
checkin = ['Arrive', 'Check_in', 'CSR', 2, 3, low]
wait = ['Arrive', 'Waiting_room', 'FlowStaff', 3, 0, low]
to_room = ['Arrive', 'To_exam_room', 'FlowStaff', 1, 0, low]
vitals = ['Exam_prep', 'Vitals_check', 'FlowStaff', 2, 0, low]
refine_complaint = ['Exam_prep', 'Refine_complaint', 'FlowStaff', 15, 15, high]
start_note = ['Exam_prep', 'Start_note', 'FlowStaff', 1, 0, low]
exam = ['Exam_provider', 'Exam', 'Doctor', np.nan, np.nan, np.nan]
checkout = ['Conclude', 'Checkout', 'CSR', 5, 5, medium]
process_flow = pd.DataFrame([checkin, wait, to_room, vitals, refine_complaint, start_note, exam, checkout],\
                            columns = cols)
process_flow['Servers'] = process_flow['Staff'].map(providers)
process_flow.loc[process_flow['Staff'] == 'CSR', 'Servers'] =\
    (process_flow.loc[process_flow['Staff'] == 'CSR', 'Servers'] / 2).astype(int)
pass_through_steps = process_flow.loc[process_flow['Time_WorstCase'] == 0]
variable_steps = process_flow.loc[process_flow['Time_WorstCase'] != 0]

In [303]:
cols = ['Type', 'Frequency', 'Time_Mean', 'Time_WorstCase', 'Perc_WorstCase']
preventative = ['Preventative', 0.2, 30, 30, low]
chronic = ['Chronic', 0.6, 30, 30, medium]
acute = ['Acute', 0.2, 15, 5, low]
base_case_types = pd.DataFrame([preventative, chronic, acute], columns = cols)

In [304]:
arrivals_day = 100
arrivals_hour = arrivals_day / 10
arrivals_minute = arrivals_hour / 60
arrivals_quarterhour = arrivals_hour / 4
arrivals_quarterhour_sigma = 0.5

In [305]:
# hours = 10
# n_periods = 60 * hours

In [306]:
processes_with_variability = variable_steps['Process'].to_list()

# Defined functions 

Issue somewhere in here

Checking in step of process -- starting 1 -> end

1. Update service time passing _(relevant for periods after first period)_
2. Move patients out of service to next step and free up servers _(relevant for periods after first period)_
3. Take new arrivals into system
4. Check number of free servers
5. Decide how many patients to move into service from wait list and from new arrivals, based on number of free servers
6. Assign patients to free servers, from wait list and/or new arrivals
7. Update wait time passing
8. Add new arrivals not assigned to free servers to waiting queue

As is check-in where problem occurs, not verifying against service time for case type as am assuming this part is ok

In [307]:
def generate_step_objects(n_servers, n_periods) :
    """Function to generate step tracking objects"""
    # Treat servers as dictionary to keep track of who is busy
    # NaN means empty server
    # If busy, dict will take form of {'Server#': n_minutes_left_service}
    dictionary_servers = {}
    for i in range(n_servers) :
        servname = 'Server' + str(i)
        dictionary_servers[servname] = np.nan
    # Treat waiting as dictionary
    # If someone waits, will be added to dictionary with form of {'Waiting#': n_minutes_waiting}
    dictionary_waiting = {}
    # Temporary tracker dictionary for service times
    dictionary_track_serve_time = {}
    # Holding lists for completed service times and completed waiting times (for measurement post-simulation)
    list_waiting_times = list()
    list_service_completed_times = list()
    # Set counter for completed service to 0
    count_service_completed = 0
    # Array for holding onto step-by-step process
    # Shape: number_of_periods x 4 -> [n_arrivals, n_being_served, n_waiting, n_completed]
    tracker = np.zeros(shape = (n_periods, 4))
    return dictionary_servers, dictionary_waiting, dictionary_track_serve_time, list_waiting_times,\
list_service_completed_times, count_service_completed, tracker

In [308]:
def generate_service_time_process(data, process, n_new_service_times = 1, skew = 0.05, n_samples = 1000) :
    """Get randomized service time based on process
    To be used only all steps in model except Exam by provider"""
    # Average service time for process
    avg = data.loc[data['Process'] == process, 'Time_Mean'].item() 
    # Worst case upper limit for process
    worstcase = data.loc[data['Process'] == process, 'Time_WorstCase'].item() 
    # % of time worst case for process
    perc_worst_case = data.loc[data['Process'] == process, 'Perc_WorstCase'].item() 
    # St Dev based on % of time worst case occurs
    std = worstcase / stats.skewnorm.ppf(1 - perc_worst_case, avg) 
    # Create distribution of 1000 samples based on condition type parameters
    dist = stats.skewnorm.rvs(skew, loc = avg, scale = std, size = n_samples)
    # Remove negative / too low results
    dist[dist < (avg * 2 / 3)] = (avg * 2 / 3)
#     dist[dist < avg / 2] = avg / 2 
    # Return number of new service times needed (will always be 1 unless exceptional circumstances)
    serve_times = np.random.choice(dist, n_new_service_times) 
    # Round service times to nearest minute
    for i, s in enumerate(serve_times) :
        serve_times[i] = round(s)
    return serve_times[0]

In [309]:
def mark_service_time(dictionary_service, count_of_completed_service, service_time_tracker_dict,\
                      list_service_times_completed) :
    """Reduce the service time left for each patient to complete step by 1
    If service time  reduced to zero, remove patient from  dictionary, move to service completed, free up server
    Return modified service time dictionary
    In parallel, track completed service times to generate avg service time estimate post-simulation"""
    # Reduce service time left to complete step
    for k, v in dictionary_service.items() :
        if np.isnan(v) :
            continue
        else :
            print ('in_mark_service_time')
            print (k, dictionary_service[k])
            dictionary_service[k] -= 1
            print (k, dictionary_service[k])
            # Count patient as completed step and free up server if service time is 0 
            if dictionary_service[k] == 0 :
                print ('in if == 0')
                print (k, dictionary_service[k])
                count_of_completed_service += 1
                dictionary_service[k] = np.nan
                print (k, dictionary_service[k])
    # For patients who are marked as completed, track the actual service time
    # completion for noting after simulation
    keys_for_removal = list()
    for k in service_time_tracker_dict.keys() :
        service_time_tracker_dict[k][0] += 1
        if service_time_tracker_dict[k][0] == service_time_tracker_dict[k][1] :
            list_service_times_completed.append(service_time_tracker_dict[k][1])
            keys_for_removal.append(k)
    for k in keys_for_removal :
        del service_time_tracker_dict[k]
    return dictionary_service, count_of_completed_service, list_service_times_completed

In [310]:
def check_servers_free(dictionary_service) :
    """Verify if servers are free (denoted by np.nan as value for server key)
    And, if free servers exist, how many"""
    count_servers_free = 0
    for k, v in dictionary_service.items() :
        if np.isnan(v) :
            count_servers_free += 1
    return count_servers_free

In [311]:
def how_many_to_move_from_where(dictionary_waiting, count_servers_free, count_n_arrivals) :
    """Determine how many people to move from wait list to service / from new arrivals to service"""
    count_from_wait_list = min(len(dictionary_waiting.keys()), count_servers_free)
    count_servers_free = count_servers_free - count_from_wait_list
    count_from_new_arrivals = min(count_n_arrivals, count_servers_free)
    return count_from_wait_list, count_from_new_arrivals

In [320]:
def move_from_wait_list_to_service(dictionary_waiting, dictionary_service, count_from_wait_list,\
                                   list_waiting_time, service_time_tracker_dict, process) :
    """Move patients from wait list to free servers
    To be preceded by: if count_from_wait_list > 0"""
    # DEBUG: REARRANGE FUNCTION TO GET LIST OF EMPTY SERVERS FIRST, THEN CHANGE SERVER
    # SEE 4_troubleshooting_service_move NOTEBOOK FOR EXPLANATION OF FIX
    empty_servers = list()
    for k, v in dictionary_service.items() :
        if np.isnan(v) :
            empty_servers.append(k)
    patients_on_wait_list = np.sort(list(dictionary_waiting.keys()))
    patients_move_to_serve = patients_on_wait_list[:count_from_wait_list]
    for i, m in enumerate(patients_move_to_serve) :
        server_for_m = empty_servers[i]
        list_waiting_time.append(dictionary_waiting[m])
        del dictionary_waiting[m]
        if process == 'Exam' :
            condition, serve_time = generate_service_time_type_condition(base_case_types)
        else :
            serve_time = generate_service_time_process(process_flow, process)
            if process == 'Refine_complaint' :
                serve_time += pass_through_steps['Time_Mean'].sum()
        dictionary_service[server_for_m] = serve_time
        unique_key = 'Service_' + str(serve_time) + '_' + str(p)
        service_time_tracker_dict[unique_key] = [0, serve_time]
    return dictionary_waiting, dictionary_service, list_waiting_time, service_time_tracker_dict

In [313]:
def move_from_arrival_to_service(dictionary_service, count_n_arrivals, count_from_new_arrivals,\
                                 service_time_tracker_dict, process) :
    """Move patients from arrivals to free servers
    To be preceded by: if count_from_new_arrivals > 0 """
    count_arrivals_placed = 0
    for k, v in dictionary_service.items() :
        if np.isnan(v) :
            if process == 'Exam' :
                condition, serve_time = generate_service_time_type_condition(base_case_types)
            else :
                serve_time = generate_service_time_process(process_flow, process)
                if process == 'Refine_complaint' :
                    serve_time += pass_through_steps['Time_Mean'].sum()
            dictionary_service[k] = serve_time
            unique_key = 'Service_' + str(serve_time) + '_' + str(p)
            service_time_tracker_dict[unique_key] = [0, serve_time]
            count_arrivals_placed += 1
            if count_arrivals_placed >= count_from_new_arrivals :
                break
    return dictionary_service, count_arrivals_placed, service_time_tracker_dict

In [314]:
def add_to_wait_list(dictionary_waiting, count_n_arrivals, count_arrivals_placed) :
    """Move patients from arrivals to wait list
    To be preceded by: if arrivals_placed < n_arrivals"""
    count_diff = count_n_arrivals - count_arrivals_placed
    wait_keys = np.sort(list(dictionary_waiting.keys()))
    for i in range(count_diff) :
        if len(wait_keys) < 1 :
            new_waitname = 'Waiting' + str(i)
            dictionary_waiting[new_waitname] = 1
        else :
            count_lastwaiter = int(wait_keys[-1][7:])
            new_waitname = 'Waiting' + str(count_lastwaiter + i + 1)
            dictionary_waiting[new_waitname] = 1
    return dictionary_waiting

Checking check-in only as first step

Verifying only in 16 period loop (full 15 period cycle +1 to ensure go all the way through) to start and setting arrivals number to be higher than server capacity (ie 5). Should be fine to troubleshoot as service time for check-in step << 15. If can verify issue here can fix across board for longer steps.

In [315]:
n_periods = 16

In [294]:
servers_check_in =\
    variable_steps.loc[variable_steps['Process'] == processes_with_variability[0], 'Servers'].item()

server_dict_check_in, waiting_dict_check_in, serve_time_track_dict_check_in,\
    waiting_time_list_check_in, service_times_completed_list_check_in, service_completed_check_in,\
    tracker_check_in = generate_step_objects(servers_check_in, n_periods)

In [295]:
for p in range(n_periods) :
    print ('\n==========\nperiod', p)
    
    # check_in step
    if p % 15 == 0 :
        n_arrivals_check_in = 8
    else :
        n_arrivals_check_in = 0
        
    server_dict_check_in, service_completed_check_in, service_times_completed_list_check_in = \
        mark_service_time(server_dict_check_in, service_completed_check_in, serve_time_track_dict_check_in,\
                          service_times_completed_list_check_in)
    n_servers_free_check_in = check_servers_free(server_dict_check_in)
    from_wait_list_check_in, from_new_arrivals_check_in = how_many_to_move_from_where(waiting_dict_check_in,\
                                                              n_servers_free_check_in, n_arrivals_check_in)
    if from_wait_list_check_in > 0 :
        waiting_dict_check_in, server_dict_check_in, waiting_time_list_check_in, serve_time_track_dict_check_in\
            = move_from_wait_list_to_service(waiting_dict_check_in, server_dict_check_in,\
                                             from_wait_list_check_in, waiting_time_list_check_in,\
                                             serve_time_track_dict_check_in, processes_with_variability[0])
    if from_new_arrivals_check_in > 0 :
        server_dict_check_in, n_arrivals_placed_check_in, serve_time_track_dict_check_in = \
            move_from_arrival_to_service(server_dict_check_in, n_arrivals_check_in, from_new_arrivals_check_in,\
                                         serve_time_track_dict_check_in, processes_with_variability[0])
    else :
        n_arrivals_placed_check_in = 0
    waiting_dict_check_in = {k:v + 1 for k, v in waiting_dict_check_in.items()}
    waiting_dict_check_in = add_to_wait_list(waiting_dict_check_in, n_arrivals_check_in,\
                                             n_arrivals_placed_check_in)
    tracker_check_in[p] = [n_arrivals_check_in, servers_check_in - \
                           [v for v in server_dict_check_in.values()].count(np.nan),\
                           len(waiting_dict_check_in.keys()), service_completed_check_in]
    print ('servers free', n_servers_free_check_in)
    print ('wl to serve', from_wait_list_check_in)
    print ('arrive to serve', from_new_arrivals_check_in)
    print (server_dict_check_in)
    print (waiting_dict_check_in)


period 0
servers free 5
wl to serve 0
arrive to serve 5
{'Server0': 2.0, 'Server1': 1.0, 'Server2': 4.0, 'Server3': 3.0, 'Server4': 3.0}
{'Waiting0': 1, 'Waiting1': 1, 'Waiting2': 1}

period 1
in_mark_service_time
Server0 2.0
Server0 1.0
in_mark_service_time
Server1 1.0
Server1 0.0
in if == 0
Server1 0.0
Server1 nan
in_mark_service_time
Server2 4.0
Server2 3.0
in_mark_service_time
Server3 3.0
Server3 2.0
in_mark_service_time
Server4 3.0
Server4 2.0
servers free 1
wl to serve 1
arrive to serve 0
{'Server0': 1.0, 'Server1': 3.0, 'Server2': 3.0, 'Server3': 2.0, 'Server4': 2.0}
{'Waiting1': 2, 'Waiting2': 2}

period 2
in_mark_service_time
Server0 1.0
Server0 0.0
in if == 0
Server0 0.0
Server0 nan
in_mark_service_time
Server1 3.0
Server1 2.0
in_mark_service_time
Server2 3.0
Server2 2.0
in_mark_service_time
Server3 2.0
Server3 1.0
in_mark_service_time
Server4 2.0
Server4 1.0
servers free 1
wl to serve 1
arrive to serve 0
{'Server0': 2.0, 'Server1': 2.0, 'Server2': 2.0, 'Server3': 1.0, 'Serv

In [296]:
print (tracker_check_in.shape)
for r in tracker_check_in :
    print (r)

(16, 4)
[8. 5. 3. 0.]
[0. 5. 2. 1.]
[0. 5. 1. 2.]
[0. 5. 0. 4.]
[0. 2. 0. 7.]
[0. 1. 0. 8.]
[0. 0. 0. 9.]
[0. 0. 0. 9.]
[0. 0. 0. 9.]
[0. 0. 0. 9.]
[0. 0. 0. 9.]
[0. 0. 0. 9.]
[0. 0. 0. 9.]
[0. 0. 0. 9.]
[0. 0. 0. 9.]
[8. 5. 3. 9.]


# Issue found

In definition of `move_from_wait_list_to_service`: Order of looping caused all empty servers to be filled.
- Before: looping through all patients to be added, then checking for empty servers, then filling those empty servers with that single patient uncontrolled because loop was looking for all empty servers -> results in all servers filled
- After: get list of empty servers first, then use `enumerate` to get an `i` for each patient to be filled into a new server, then ensure only filling that specific server by a) removing extra for loop that was cyling through all `key`-`value` paris that were `np.nan` within the loop for each patient to move, then b) using `empty_servers[i]` to only fill an individual server within that loop.

Leaving elements above for reference in future. Below is fresh run with problem fixed, added lines noted in documentation in `def move_from_wait_list_to_service`

In [298]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [316]:
servers_check_in =\
    variable_steps.loc[variable_steps['Process'] == processes_with_variability[0], 'Servers'].item()

server_dict_check_in, waiting_dict_check_in, serve_time_track_dict_check_in,\
    waiting_time_list_check_in, service_times_completed_list_check_in, service_completed_check_in,\
    tracker_check_in = generate_step_objects(servers_check_in, n_periods)

In [317]:
for p in range(n_periods) :
    print ('\n==========\nperiod', p)
    
    # check_in step
    if p % 15 == 0 :
        n_arrivals_check_in = 8
    else :
        n_arrivals_check_in = 0
        
    server_dict_check_in, service_completed_check_in, service_times_completed_list_check_in = \
        mark_service_time(server_dict_check_in, service_completed_check_in, serve_time_track_dict_check_in,\
                          service_times_completed_list_check_in)
    n_servers_free_check_in = check_servers_free(server_dict_check_in)
    from_wait_list_check_in, from_new_arrivals_check_in = how_many_to_move_from_where(waiting_dict_check_in,\
                                                              n_servers_free_check_in, n_arrivals_check_in)
    if from_wait_list_check_in > 0 :
        waiting_dict_check_in, server_dict_check_in, waiting_time_list_check_in, serve_time_track_dict_check_in\
            = move_from_wait_list_to_service(waiting_dict_check_in, server_dict_check_in,\
                                             from_wait_list_check_in, waiting_time_list_check_in,\
                                             serve_time_track_dict_check_in, processes_with_variability[0])
    if from_new_arrivals_check_in > 0 :
        server_dict_check_in, n_arrivals_placed_check_in, serve_time_track_dict_check_in = \
            move_from_arrival_to_service(server_dict_check_in, n_arrivals_check_in, from_new_arrivals_check_in,\
                                         serve_time_track_dict_check_in, processes_with_variability[0])
    else :
        n_arrivals_placed_check_in = 0
    waiting_dict_check_in = {k:v + 1 for k, v in waiting_dict_check_in.items()}
    waiting_dict_check_in = add_to_wait_list(waiting_dict_check_in, n_arrivals_check_in,\
                                             n_arrivals_placed_check_in)
    tracker_check_in[p] = [n_arrivals_check_in, servers_check_in - \
                           [v for v in server_dict_check_in.values()].count(np.nan),\
                           len(waiting_dict_check_in.keys()), service_completed_check_in]
    print ('servers free', n_servers_free_check_in)
    print ('wl to serve', from_wait_list_check_in)
    print ('arrive to serve', from_new_arrivals_check_in)
    print (server_dict_check_in)
    print (waiting_dict_check_in)


period 0
servers free 5
wl to serve 0
arrive to serve 5
{'Server0': 5.0, 'Server1': 3.0, 'Server2': 2.0, 'Server3': 5.0, 'Server4': 2.0}
{'Waiting0': 1, 'Waiting1': 1, 'Waiting2': 1}

period 1
in_mark_service_time
Server0 5.0
Server0 4.0
in_mark_service_time
Server1 3.0
Server1 2.0
in_mark_service_time
Server2 2.0
Server2 1.0
in_mark_service_time
Server3 5.0
Server3 4.0
in_mark_service_time
Server4 2.0
Server4 1.0
servers free 0
wl to serve 0
arrive to serve 0
{'Server0': 4.0, 'Server1': 2.0, 'Server2': 1.0, 'Server3': 4.0, 'Server4': 1.0}
{'Waiting0': 2, 'Waiting1': 2, 'Waiting2': 2}

period 2
in_mark_service_time
Server0 4.0
Server0 3.0
in_mark_service_time
Server1 2.0
Server1 1.0
in_mark_service_time
Server2 1.0
Server2 0.0
in if == 0
Server2 0.0
Server2 nan
in_mark_service_time
Server3 4.0
Server3 3.0
in_mark_service_time
Server4 1.0
Server4 0.0
in if == 0
Server4 0.0
Server4 nan
servers free 2
wl to serve 2
arrive to serve 0
{'Server0': 3.0, 'Server1': 1.0, 'Server2': 5.0, 'Serve

In [318]:
print (tracker_check_in.shape)
for r in tracker_check_in :
    print (r)

(16, 4)
[8. 5. 3. 0.]
[0. 5. 3. 0.]
[0. 5. 1. 2.]
[0. 5. 0. 3.]
[0. 5. 0. 3.]
[0. 3. 0. 5.]
[0. 1. 0. 7.]
[0. 0. 0. 8.]
[0. 0. 0. 8.]
[0. 0. 0. 8.]
[0. 0. 0. 8.]
[0. 0. 0. 8.]
[0. 0. 0. 8.]
[0. 0. 0. 8.]
[0. 0. 0. 8.]
[8. 5. 3. 8.]


Bad code below for documentation's sake

In [319]:
def move_from_wait_list_to_service(dictionary_waiting, dictionary_service, count_from_wait_list,\
                                   list_waiting_time, service_time_tracker_dict, process) :
    """Move patients from wait list to free servers
    To be preceded by: if count_from_wait_list > 0"""
    patients_on_wait_list = np.sort(list(dictionary_waiting.keys()))
    patients_move_to_serve = patients_on_wait_list[:count_from_wait_list]
    for m in patients_move_to_serve :
        list_waiting_time.append(dictionary_waiting[m])
        del dictionary_waiting[m]
        for k, v in dictionary_service.items() :
            if np.isnan(v) :
                if process == 'Exam' :
                    condition, serve_time = generate_service_time_type_condition(base_case_types)
                else :
                    serve_time = generate_service_time_process(process_flow, process)
                    if process == 'Refine_complaint' :
                        serve_time += pass_through_steps['Time_Mean'].sum()
                dictionary_service[k] = serve_time
                unique_key = 'Service_' + str(serve_time) + '_' + str(p)
                service_time_tracker_dict[unique_key] = [0, serve_time]
    return dictionary_waiting, dictionary_service, list_waiting_time, service_time_tracker_dict